# Baselines for Commons Upload Wizard Improvements

[T337466](https://phabricator.wikimedia.org/T337466)

The goal of this task is to calculate baselines success metrics and goals for Commons upload wizard improvements. 

In this notebook, we want include the baselines for the follwoing metrics:
- Total number of upload media within a month through upload wizard (filter by own work and not own work)
- Total number of filed deletion requests and total number of speedy deletions within a month (filter by own work and not own work)
- Deletion rate of upload media (filter bu own work and not own work)
- Time to deletions

In [ ]:
import re

from wmfdata import hive, mariadb, spark
import wmfdata 

import math
import pandas as pd
import numpy as np

from datetime import datetime, timedelta, date

In [3]:
spark_session  = wmfdata.spark.create_custom_session(
    master='yarn',
    spark_config={
        'spark.driver.memory': '32g',
        'spark.driver.cores': '4',
        'spark.executor.memory': '32g',
        'spark.executor.core': '8',
        'spark.dynamicAllocation.maxExecutors': '256'
    }
)

SPARK_HOME: /usr/lib/spark3
Using Hadoop client lib jars at 3.2.0, provided by Spark.
PYSPARK_PYTHON=/opt/conda-analytics/bin/python3


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/26 07:16:14 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/06/26 07:16:14 WARN Utils: Service 'sparkDriver' could not bind on port 12000. Attempting port 12001.
23/06/26 07:16:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/06/26 07:16:23 WARN Utils: Service 'org.apache.spark.network.netty.NettyBlockTransferService' could not bind on port 13000. Attempting port 13001.
23/06/26 07:16:23 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!


In [4]:
snapshot = '2023-05'  
start_date = '2023-01-01'
end_date = '2023-06-01'

## File Uploads

Get file uploads data, we will exclude bot and mobile uploads here. 

In `mediawiki_history` table, for deleted pages, the event_comment and page_is_redirect fields are both NULL. In this case, in order to get file uploads that are deleted and their event comments, we need to join with `mediawiki_logging` and `mediawiki_private_comment` table.

In [5]:
# get uploads that are not deleted 
uploads_ud_query = """
SELECT
   event_timestamp,
   page_id,
   CASE WHEN ARRAY_CONTAINS(revision_tags, 'uploadwizard') THEN true ELSE false END AS upload_wizard,
   CASE WHEN ARRAY_CONTAINS(revision_tags, 'ios app edit') 
            OR ARRAY_CONTAINS(revision_tags, 'android app edit') 
            OR ARRAY_CONTAINS(revision_tags, 'mobile app edit') 
            OR ARRAY_CONTAINS(revision_tags, 'mobile web edit') THEN 'mobile'
        ELSE 'other'
    END AS platform,
    CASE WHEN LOWER(event_comment) LIKE '%own work%' THEN true ELSE false END AS own_work,
    page_is_deleted
FROM wmf.mediawiki_history
WHERE snapshot = '{mw_snapshot}' 
    AND event_timestamp >= '{start_date}'
    AND event_timestamp < '{end_date}' 
    AND event_entity = 'revision'
    AND event_type = 'create' 
    AND page_namespace=6
    AND revision_parent_id = 0
    AND NOT page_is_redirect 
    AND NOT page_is_deleted
    AND wiki_db = 'commonswiki'
    AND SIZE(event_user_is_bot_by) <= 0
    AND SIZE(event_user_is_bot_by_historical) <= 0
"""

In [6]:
upload_ud_data = spark.run( 
        uploads_ud_query.format(
          start_date = start_date,
          end_date = end_date,
          mw_snapshot = snapshot
        )
    )

23/06/26 07:16:28 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
23/06/26 07:17:20 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 193 for reason Container container_e85_1687442676858_18332_01_000384 on host: an-worker1098.eqiad.wmnet was preempted.
23/06/26 07:17:20 ERROR YarnScheduler: Lost executor 193 on an-worker1098.eqiad.wmnet: Container container_e85_1687442676858_18332_01_000384 on host: an-worker1098.eqiad.wmnet was preempted.
23/06/26 07:17:38 ERROR YarnScheduler: Lost executor 197 on an-worker1098.eqiad.wmnet: Container container_e85_1687442676858_18332_01_000389 on host: an-worker1098.eqiad.wmnet was preempted.
23/06/26 07:17:38 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 197 for reason Container container_e85_1687442676858_18332_01_000389 on host: an-worker1098.eqiad.wmnet was preempted.
23/06/26 

In [7]:
# get uploads that are deleted 
uploads_d_query = """
SELECT
   event_timestamp,
   page_id,
   CASE WHEN ARRAY_CONTAINS(revision_tags, 'uploadwizard') THEN true ELSE false END AS upload_wizard,
   CASE WHEN ARRAY_CONTAINS(revision_tags, 'ios app edit') 
            OR ARRAY_CONTAINS(revision_tags, 'android app edit') 
            OR ARRAY_CONTAINS(revision_tags, 'mobile app edit') 
            OR ARRAY_CONTAINS(revision_tags, 'mobile web edit') THEN 'mobile'
        ELSE 'other'
    END AS platform,
    CASE WHEN LOWER(c.comment_text) LIKE '%own work%' THEN true ELSE false END AS own_work,
    page_is_deleted
FROM wmf.mediawiki_history mh 
    LEFT JOIN wmf_raw.mediawiki_logging l ON mh.page_id = l.log_page
    LEFT JOIN wmf_raw.mediawiki_private_comment c  ON l.log_comment_id = c.comment_id
WHERE mh.snapshot = '{mw_snapshot}' 
    AND c.snapshot = '{mw_snapshot}'
    AND l.snapshot = '{mw_snapshot}'
    AND event_timestamp >= '{start_date}'
    AND event_timestamp < '{end_date}' 
    AND event_entity = 'revision' 
    AND event_type = 'create' 
    AND page_namespace=6
    AND log_namespace = 6
    AND log_type = 'upload'
    AND log_action = 'upload'
    AND revision_parent_id = 0
    AND page_is_deleted
    AND mh.wiki_db = 'commonswiki'
    AND c.wiki_db = 'commonswiki'
    AND l.wiki_db = 'commonswiki'
    AND SIZE(event_user_is_bot_by) <= 0
    AND SIZE(event_user_is_bot_by_historical) <= 0
"""

In [8]:
upload_d_data = spark.run( 
        uploads_d_query.format(
          start_date = start_date,
          end_date = end_date,
          mw_snapshot = snapshot
        )
    )

In [9]:
upload_data = upload_d_data.append(upload_ud_data)

/tmp/ipykernel_4374/2623581098.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  upload_data = upload_d_data.append(upload_ud_data)


In [10]:
# store in global temp view
uploads_sdf = spark_session.createDataFrame(upload_data)
uploads_sdf.createGlobalTempView("upload_data")

## Deletions

In [19]:
deletion_query = """
SELECT
   event_timestamp AS deletion_ts,
   page_id,
   CASE WHEN LOWER(c.comment_text) LIKE '%speedy%' THEN true ELSE false END AS speedy_deletion
FROM wmf.mediawiki_history mh
    LEFT JOIN wmf_raw.mediawiki_logging l ON mh.page_id = l.log_page
    LEFT JOIN wmf_raw.mediawiki_private_comment c  ON l.log_comment_id = c.comment_id
WHERE mh.snapshot = '{mw_snapshot}' 
    AND c.snapshot = '{mw_snapshot}'
    AND l.snapshot = '{mw_snapshot}'
    AND event_timestamp >= '{start_date}'
    AND event_timestamp < '{end_date}' 
    AND event_entity = 'revision' 
    AND event_type = 'create' 
    AND log_type = 'delete'
    AND log_action = 'delete'
    AND page_namespace=6
    AND page_is_redirect IS NULL
    AND page_is_deleted
    AND mh.wiki_db = 'commonswiki'
    AND c.wiki_db = 'commonswiki'
    AND l.wiki_db = 'commonswiki'
    AND SIZE(event_user_is_bot_by) <= 0
    AND SIZE(event_user_is_bot_by_historical) <= 0
"""

In [20]:
deletion_data = spark.run( 
        deletion_query.format(
          start_date = start_date,
          end_date = end_date,
          mw_snapshot = snapshot
        )
    )

In [21]:
# store in global temp view
deletion_sdf = spark_session.createDataFrame(deletion_data)
deletion_sdf.createGlobalTempView("deletion_data")

## File Uploads Metrics

File uploads in April and May 2023

In [11]:
spark.run("""
SELECT 
    trunc(event_timestamp, "MONTH") as month,
    upload_wizard, own_work, 
    COUNT(DISTINCT(page_id)) AS uploads,
    COUNT(DISTINCT(page_id)) * 100.0 / SUM(COUNT(DISTINCT(page_id))) OVER (PARTITION BY trunc(event_timestamp, "MONTH")) AS pct
FROM global_temp.upload_data
WHERE platform = 'other'
  AND event_timestamp >= '2023-04-01'
  AND event_timestamp < '2023-06-01' 
GROUP BY trunc(event_timestamp, "MONTH"),upload_wizard, own_work
""")

23/06/26 07:22:25 WARN TaskSetManager: Stage 6 contains a task of very large size (60351 KiB). The maximum recommended task size is 1000 KiB.


,month,upload_wizard,own_work,uploads,pct
0,2023-04-01,True,True,215598,37.19348728405668
1,2023-04-01,True,False,88303,15.23342752550607
2,2023-04-01,False,False,275696,47.56118178399285
3,2023-04-01,False,True,69,0.01190340644440
4,2023-05-01,True,True,234775,40.13007768765971
5,2023-05-01,False,False,271025,46.32628817079320
6,2023-05-01,False,True,158,0.02700693120924
7,2023-05-01,True,False,79077,13.51662721033784


In April 2023, there are 579,666 uploads on desktop in Commons excluding bot uploads. 303,901 (52.3%) of uploads are from upload wizard; and 276,038 (47.6%) are from other upload method. 
In all the uploads from upload wizard, 71.1% are own work, and 28.9% are not own work. 

In May 2023, there are 585,035 uploads on desktop in Commons excluding bot uploads. 313,852 (53.6%) of uploads are from upload wizard; and 271,183 (46.4%) are from other upload method. 
In all the uploads from upload wizard, 74.6% are own work, and 25.2% are not own work. 

While for both months, from other upload method, it's hard to define the files are own work or not, because not as the upload wizard, the edit comments are not standard or in other languages. We can only detect 0.02% of the files are own work. 

## Deletion Request Metrics

Deletions in April and May 2023. 
Note that, we use  edit comment to determine whether deletions are speedy deletions or not. But a large number of comments are empty or not standard. As a result, it is possible that the actual speedy deletion numbers are greater than the data we calculated. 

In [22]:
spark.run("""
SELECT 
    trunc(event_timestamp, "MONTH") AS month,
    upload_wizard, 
    own_work, 
    COUNT(DISTINCT(u.page_id)) AS deletion_request,
    COUNT(DISTINCT CASE WHEN speedy_deletion THEN u.page_id END) AS sd_request
FROM global_temp.upload_data u 
    LEFT JOIN global_temp.deletion_data d ON u.page_id = d.page_id
WHERE platform = 'other'
  AND event_timestamp >= '2023-04-01'
  AND event_timestamp < '2023-06-01' 
  AND page_is_deleted
GROUP BY trunc(event_timestamp, "MONTH"),upload_wizard, own_work
""")

23/06/26 07:44:39 WARN TaskSetManager: Stage 29 contains a task of very large size (60351 KiB). The maximum recommended task size is 1000 KiB.


,month,upload_wizard,own_work,deletion_request,sd_request
0,2023-05-01,True,True,4993,96
1,2023-05-01,False,True,1,0
2,2023-04-01,True,True,4847,93
3,2023-04-01,True,False,5076,77
4,2023-05-01,False,False,5114,79
5,2023-05-01,True,False,2966,81
6,2023-04-01,False,False,6359,101
7,2023-04-01,False,True,3,0


In April 2023, there are in total 16,285 deletion requests, 271 of them are speedy deletions. 60.9% of them are uploads from upload wizard, and 38.9% of them are from other upload methods.
In upload wizard uploads, there are 9,923 deletions. 4,847 (48.8%) are own work and 5,076 (51.2%) are not own work. 170 of them are speedy deletions. 

In May 2023, there are in total 13,074 deletion requests, 256 of them are speedy deletions. 60.9% of them are uploads from upload wizard, and 39% of them are from other upload methods.
In upload wizard uploads, there are 7,959 deletions. 4,993 (63%) are own work and 2966 (37%) are not own work. 177 of them are speedy deletions.

## Deletion Rate

Deletion rate MoM excluding speedy deletions

In [28]:
spark.run("""
WITH uploads AS (
    SELECT 
        TRUNC(event_timestamp, "MONTH") as month,
        COUNT(DISTINCT page_id) AS all_upload,
        COUNT(DISTINCT CASE WHEN upload_wizard AND own_work THEN page_id END) AS uw_own_upload,
        COUNT(DISTINCT CASE WHEN upload_wizard AND NOT own_work THEN page_id END) AS uw_not_own_upload,
        COUNT(DISTINCT CASE WHEN NOT upload_wizard THEN page_id END) AS not_uw_upload
    FROM global_temp.upload_data
    WHERE platform = 'other'
      AND event_timestamp >= '2023-01-01'
      AND event_timestamp < '2023-06-01' 
    GROUP BY TRUNC(event_timestamp, "MONTH")
), 
deletions AS (
    SELECT 
        TRUNC(event_timestamp, "MONTH") AS month,
        COUNT(DISTINCT u.page_id) AS all_dele,
        COUNT(DISTINCT CASE WHEN upload_wizard AND own_work THEN u.page_id END) AS uw_own_dele,
        COUNT(DISTINCT CASE WHEN upload_wizard AND NOT own_work THEN u.page_id END) AS uw_not_own_dele,
        COUNT(DISTINCT CASE WHEN NOT upload_wizard THEN u.page_id END) AS not_uw_dele
    FROM global_temp.upload_data u 
        LEFT JOIN global_temp.deletion_data d ON u.page_id = d.page_id
    WHERE platform = 'other'
      AND event_timestamp >= '2023-01-01'
      AND event_timestamp < '2023-06-01' 
      AND page_is_deleted
      AND NOT speedy_deletion
    GROUP BY TRUNC(event_timestamp, "MONTH")
)

SELECT
    d.month,
    all_dele/all_upload * 100 AS all_dr,
    (uw_own_dele + uw_not_own_dele)/ (uw_own_upload +uw_not_own_upload) * 100 AS uw_all_dr,
    uw_own_dele/uw_own_upload * 100 AS uw_own_dr,
    uw_not_own_dele/uw_not_own_upload * 100 AS uw_not_own_dr,
    not_uw_dele/not_uw_upload * 100 AS not_uw_dr
FROM uploads u JOIN deletions d ON d.month=u.month
ORDER BY month
""")

23/06/27 06:02:53 WARN TaskSetManager: Stage 41 contains a task of very large size (60351 KiB). The maximum recommended task size is 1000 KiB.
23/06/27 06:02:54 WARN TaskSetManager: Stage 43 contains a task of very large size (60351 KiB). The maximum recommended task size is 1000 KiB.


,month,all_dr,uw_all_dr,uw_own_dr,uw_not_own_dr,not_uw_dr
0,2023-01-01,4.687209,4.777680,3.596144,7.841766,4.595278
1,2023-02-01,4.583632,4.300957,3.625446,5.966741,4.917870
2,2023-03-01,3.363374,3.830104,2.732612,6.390900,2.903313
3,2023-04-01,2.762635,3.209269,2.205030,5.661189,2.270411
4,2023-05-01,2.191051,2.479513,2.085827,3.648343,1.857049


In April, the overall deletion rate is 2.76%. the upload wizard uploads (3.21%) has higher deletion rate than uploads through other methods (2.27%). 
Among the uploads through upload wizard, own work 2.21% has lower deletion rate than not own work (5.66%). 

In May, the overrall deletion rate is 2.19%. the upload wizard uploads (2.09%) has higher deletion rate than uploads through other methods (1.85%). 
Among the uploads through upload wizard, own work 2.09% has lower deletion rate than not own work (3.65%). 

From the MoM deletion rates, we can see the deletion rates increasing over time. 

Deletion rate MoM with only speedy deletions

In [29]:
spark.run("""
WITH uploads AS (
    SELECT 
        TRUNC(event_timestamp, "MONTH") as month,
        COUNT(DISTINCT page_id) AS all_upload,
        COUNT(DISTINCT CASE WHEN upload_wizard AND own_work THEN page_id END) AS uw_own_upload,
        COUNT(DISTINCT CASE WHEN upload_wizard AND NOT own_work THEN page_id END) AS uw_not_own_upload,
        COUNT(DISTINCT CASE WHEN NOT upload_wizard THEN page_id END) AS not_uw_upload
    FROM global_temp.upload_data
    WHERE platform = 'other'
      AND event_timestamp >= '2023-01-01'
      AND event_timestamp < '2023-06-01' 
    GROUP BY TRUNC(event_timestamp, "MONTH")
), 
deletions AS (
    SELECT 
        TRUNC(event_timestamp, "MONTH") AS month,
        COUNT(DISTINCT u.page_id) AS all_dele,
        COUNT(DISTINCT CASE WHEN upload_wizard AND own_work THEN u.page_id END) AS uw_own_dele,
        COUNT(DISTINCT CASE WHEN upload_wizard AND NOT own_work THEN u.page_id END) AS uw_not_own_dele,
        COUNT(DISTINCT CASE WHEN NOT upload_wizard THEN u.page_id END) AS not_uw_dele
    FROM global_temp.upload_data u 
        LEFT JOIN global_temp.deletion_data d ON u.page_id = d.page_id
    WHERE platform = 'other'
      AND event_timestamp >= '2023-01-01'
      AND event_timestamp < '2023-06-01' 
      AND page_is_deleted
      AND speedy_deletion
    GROUP BY TRUNC(event_timestamp, "MONTH")
)

SELECT
    d.month,
    all_dele/all_upload * 100 AS all_dr,
    (uw_own_dele + uw_not_own_dele)/ (uw_own_upload +uw_not_own_upload) * 100 AS uw_all_dr,
    uw_own_dele/uw_own_upload * 100 AS uw_own_dr,
    uw_not_own_dele/uw_not_own_upload * 100 AS uw_not_own_dr,
    not_uw_dele/not_uw_upload * 100 AS not_uw_dr
FROM uploads u JOIN deletions d ON d.month=u.month
ORDER BY month
""")

23/06/27 07:35:45 WARN TaskSetManager: Stage 56 contains a task of very large size (60351 KiB). The maximum recommended task size is 1000 KiB.
23/06/27 07:35:52 WARN TaskSetManager: Stage 59 contains a task of very large size (60351 KiB). The maximum recommended task size is 1000 KiB.


,month,all_dr,uw_all_dr,uw_own_dr,uw_not_own_dr,not_uw_dr
0,2023-01-01,0.024917,0.030364,0.028588,0.034970,0.019383
1,2023-02-01,0.010522,0.010662,0.010704,0.010558,0.010356
2,2023-03-01,0.015307,0.022955,0.015662,0.039972,0.007767
3,2023-04-01,0.046751,0.055939,0.043136,0.087200,0.036625
4,2023-05-01,0.043759,0.056396,0.040890,0.102432,0.029132


## Time to Deletion